In [ ]:
import torch
from torch import nn
from torch import save
from torch import load
from torch.optim import Adam
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from PIL import Image

In [ ]:
train = datasets.MNIST(root="dataset/", download=True, train=True, transform=ToTensor())
test = datasets.MNIST(root="dataset/", download=True, train=False, transform=ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [ ]:
#batch size = 64
data_train = DataLoader(train,64)
data_test = DataLoader(test,64) 

In [ ]:
# neural network 
class neural (nn.Module):
  def __init__(self):
    super().__init__()
    self.network = nn.Sequential(
        # 1 because blac and white , 28x28
        nn.Conv2d(1,32, (3,3)), nn.ReLU(), nn.Conv2d(32,64,(3,3)),nn.ReLU(),nn.Conv2d(64,128,(3,3)),nn.ReLU(),nn.Conv2d(128,128,(3,3)),nn.ReLU(),nn.Flatten(),
        nn.Linear(128*(28-8)*(28-8),10)
    )
  def forward(self,temp_model):
    return self.network(temp_model)

In [ ]:
net = neural().to('cuda')
optimizer = Adam(net.parameters(),lr = 0.001)
loss = nn.CrossEntropyLoss()


In [ ]:
#training
#epochs 20
for epoch in range(0,10):
  for data in data_train:
    #unpacking
      x,y = data
    # to device
      x = x.to('cuda')
      y = y.to('cuda')
    # prediction
      pred = net(x)
    # loss calculation
      lc = loss(pred,y)
    #backprob
    #existing gradients
      optimizer.zero_grad()
    # calculating gradients
      lc.backward()
    # gradient descent
      optimizer.step()
  print('Epoch: ',epoch)
  print('Loss:',lc.item())

Epoch:  0
Loss: 0.037089910358190536
Epoch:  1
Loss: 0.0014385367976501584
Epoch:  2
Loss: 0.0003671291924547404
Epoch:  3
Loss: 9.65931176324375e-05
Epoch:  4
Loss: 0.0020371421705931425
Epoch:  5
Loss: 2.352965202589985e-05
Epoch:  6
Loss: 0.00010308861237717792
Epoch:  7
Loss: 6.340053005260415e-06
Epoch:  8
Loss: 3.401022468096926e-06
Epoch:  9
Loss: 1.400873952661641e-05


In [ ]:
num_correct = 0

num_samples = 0
net.eval()

with torch.no_grad():
    for x, y in data_train:
        x = x.to('cuda')
        y = y.to('cuda')

        scores = net(x)
        _, predictions = scores.max(1)
        num_correct += (predictions == y).sum()
        num_samples += predictions.size(0)


    net.train()
    accuracy = num_correct/num_samples


print("Accuracy on training set:",accuracy*100)

Accuracy on training set: tensor(99.8200, device='cuda:0')


In [ ]:
with open('state.pt', 'wb') as f: 
        save(net.state_dict(), f) 

with open('state.pt', 'rb') as f: 
        net.load_state_dict(load(f))  

print(torch.argmax(net(ToTensor()(Image.open('/content/test_image_4.jpg') ).unsqueeze(0).to('cuda'))))

tensor(2, device='cuda:0')
